In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import json

print("🚀 Starting Fragile Watermarking System...")

🚀 Starting Fragile Watermarking System...


In [6]:
# 1. Core Functions
def load_and_preprocess_image(image_path):
    """Load and preprocess image for watermarking"""
    try:
        img = Image.open(image_path)
        img_array = np.array(img)
        print(f"✅ Image loaded: {img_array.shape}")
        return img_array, 'color' if len(img_array.shape) == 3 else 'grayscale'
    except Exception as e:
        print(f"❌ Error loading image: {e}")
        return None, None

def text_to_binary(text):
    """Convert text to binary"""
    return ''.join(format(ord(char), '08b') for char in text)

def binary_to_text(binary_str):
    """Convert binary back to text"""
    text = ''
    for i in range(0, len(binary_str), 8):
        byte = binary_str[i:i+8]
        if len(byte) == 8:
            text += chr(int(byte, 2))
    return text

def embed_watermark_lsb(image_array, watermark_text, mode='color'):
    """Embed watermark using LSB"""
    binary_watermark = text_to_binary(watermark_text) + '00000000'  # Null terminator
    watermark_length = len(binary_watermark)
    
    watermarked_image = image_array.copy()
    
    if mode == 'color':
        flat_pixels = watermarked_image.reshape(-1)
    else:
        flat_pixels = watermarked_image.reshape(-1)
    
    if watermark_length > len(flat_pixels):
        raise ValueError("❌ Watermark too large for image!")
    
    embedding_map = np.zeros_like(flat_pixels, dtype=bool)
    
    # Embed the watermark
    for i in range(watermark_length):
        flat_pixels[i] = (flat_pixels[i] & 0xFE) | int(binary_watermark[i])
        embedding_map[i] = True
    
    if mode == 'color':
        watermarked_image = flat_pixels.reshape(image_array.shape)
    else:
        watermarked_image = flat_pixels.reshape(image_array.shape)
    
    return watermarked_image, watermark_length

def extract_watermark_lsb(image_array, watermark_length, mode='color'):
    """Extract watermark from LSB"""
    if mode == 'color':
        flat_pixels = image_array.reshape(-1)
    else:
        flat_pixels = image_array.reshape(-1)
    
    binary_watermark = ''.join(str(flat_pixels[i] & 1) for i in range(watermark_length))
    return binary_to_text(binary_watermark)

def calculate_quality_metrics(original, watermarked):
    """Calculate image quality metrics"""
    mse = np.mean((original.astype(float) - watermarked.astype(float)) ** 2)
    psnr = 20 * np.log10(255.0 / np.sqrt(mse)) if mse > 0 else float('inf')
    
    return {'MSE': mse, 'PSNR': psnr}

def save_watermark_info(watermark_text, watermark_length, original_path, output_path):
    """Save watermark information to a JSON file for verification"""
    info = {
        'watermark_text': watermark_text,
        'watermark_length': watermark_length,
        'original_image': original_path,
        'watermarked_image': output_path,
        'timestamp': np.datetime64('now').astype(str)
    }
    
    info_path = output_path.replace('.png', '_watermark_info.json')
    with open(info_path, 'w') as f:
        json.dump(info, f, indent=2)
    
    return info_path

def load_watermark_info(info_path):
    """Load watermark information from JSON file"""
    try:
        with open(info_path, 'r') as f:
            return json.load(f)
    except:
        return None

In [7]:
# 2. Embedding Interface
def create_embedding_interface():
    """Interface for embedding watermarks"""
    
    # Create widgets
    upload = widgets.FileUpload(
        description='📁 Upload Image',
        accept='.jpg,.jpeg,.png,.bmp,.tiff',
        multiple=False
    )
    
    watermark_text = widgets.Textarea(
        value='© Copyright 2024',
        description='💧 Watermark:',
        layout=widgets.Layout(width='400px', height='60px')
    )
    
    embed_btn = widgets.Button(
        description='🔒 Embed Watermark',
        button_style='success',
        icon='lock'
    )
    
    output_area = widgets.Output()
    
    def embed_watermark(b):
        with output_area:
            clear_output()
            print("⏳ Embedding watermark...")
            
            content = get_uploaded_content(upload)
            if content is None:
                print("❌ Please upload an image first!")
                return
                
            try:
                # Save uploaded image
                temp_path = 'original_image.png'
                with open(temp_path, 'wb') as f:
                    f.write(content)
                
                # Load image
                original_image, image_mode = load_and_preprocess_image(temp_path)
                if original_image is None:
                    return
                
                print(f"🎨 Processing {image_mode} image...")
                
                # Embed watermark
                watermarked, wm_length = embed_watermark_lsb(
                    original_image, watermark_text.value, image_mode
                )
                
                # Display results
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
                
                if image_mode == 'color':
                    ax1.imshow(original_image)
                    ax2.imshow(watermarked)
                else:
                    ax1.imshow(original_image, cmap='gray')
                    ax2.imshow(watermarked, cmap='gray')
                
                ax1.set_title('📷 Original Image')
                ax2.set_title('🔒 Watermarked Image')
                ax1.axis('off')
                ax2.axis('off')
                plt.tight_layout()
                plt.show()
                
                # Calculate quality metrics
                metrics_result = calculate_quality_metrics(original_image, watermarked)
                print("\n📊 QUALITY METRICS:")
                print(f"   PSNR: {metrics_result['PSNR']:.2f} dB (Higher = Better)")
                print(f"   MSE:  {metrics_result['MSE']:.2f} (Lower = Better)")
                
                # Save watermarked image
                output_path = 'watermarked_image.png'
                result_img = Image.fromarray(watermarked.astype(np.uint8))
                if image_mode == 'grayscale':
                    result_img = result_img.convert('L')
                result_img.save(output_path)
                
                # Save watermark info for verification
                info_path = save_watermark_info(
                    watermark_text.value, wm_length, temp_path, output_path
                )
                
                print(f"\n💾 SAVED FILES:")
                print(f"   Watermarked image: {output_path}")
                print(f"   Watermark info: {info_path}")
                
                print(f"\n✅ EMBEDDING COMPLETE!")
                print("   You can now use the verification tab to check the watermark.")
                
            except Exception as e:
                print(f"❌ Error during embedding: {str(e)}")
            finally:
                # Cleanup
                if os.path.exists(temp_path):
                    os.remove(temp_path)
    
    embed_btn.on_click(embed_watermark)
    
    return widgets.VBox([
        widgets.HTML("<h2>🔒 Phase 1: Embed Watermark</h2>"),
        widgets.HTML("<p>Upload an image and embed your copyright watermark</p>"),
        upload,
        watermark_text,
        embed_btn,
        output_area
    ])

In [8]:
# 3. Verification Interface
def create_verification_interface():
    """Interface for verifying watermarks"""
    
    # Create widgets
    verify_upload = widgets.FileUpload(
        description='📁 Upload Watermarked Image',
        accept='.jpg,.jpeg,.png,.bmp,.tiff',
        multiple=False
    )
    
    info_upload = widgets.FileUpload(
        description='📁 Upload Watermark Info (JSON)',
        accept='.json',
        multiple=False
    )
    
    manual_watermark = widgets.Textarea(
        value='',
        description='🔍 Watermark to verify:',
        layout=widgets.Layout(width='400px', height='60px'),
        placeholder='Enter the expected watermark text (if no JSON file)'
    )
    
    verify_btn = widgets.Button(
        description='🔍 Verify Watermark',
        button_style='info',
        icon='search'
    )
    
    output_area = widgets.Output()
    
    def verify_watermark(b):
        with output_area:
            clear_output()
            print("⏳ Verifying watermark...")
            
            image_content = get_uploaded_content(verify_upload)
            if image_content is None:
                print("❌ Please upload a watermarked image!")
                return
            
            try:
                # Save uploaded image
                temp_image_path = 'verify_image.png'
                with open(temp_image_path, 'wb') as f:
                    f.write(image_content)
                
                # Load image for verification
                verify_image, image_mode = load_and_preprocess_image(temp_image_path)
                if verify_image is None:
                    return
                
                watermark_info = None
                expected_watermark = manual_watermark.value
                watermark_length = None
                
                # Try to load watermark info from JSON
                info_content = get_uploaded_content(info_upload)
                if info_content:
                    temp_info_path = 'watermark_info.json'
                    with open(temp_info_path, 'wb') as f:
                        f.write(info_content)
                    watermark_info = load_watermark_info(temp_info_path)
                    
                    if watermark_info:
                        expected_watermark = watermark_info['watermark_text']
                        watermark_length = watermark_info['watermark_length']
                        print("✅ Loaded watermark info from JSON file")
                
                # If no JSON, use manual input
                if not expected_watermark:
                    print("❌ No watermark text provided! Use JSON file or manual entry.")
                    return
                
                # Calculate watermark length if not provided
                if watermark_length is None:
                    watermark_length = len(text_to_binary(expected_watermark)) + 8  # +8 for null terminator
                
                # Extract watermark
                extracted_watermark = extract_watermark_lsb(verify_image, watermark_length, image_mode)
                
                # Display verification results
                print(f"\n🔍 VERIFICATION RESULTS:")
                print(f"   Expected: {expected_watermark}")
                print(f"   Extracted: {extracted_watermark}")
                
                # Check for exact match first, then partial match
                if expected_watermark in extracted_watermark:
                    match_percentage = (len(expected_watermark) / len(extracted_watermark.strip())) * 100
                    print(f"   ✅ MATCH FOUND: {match_percentage:.1f}%")
                    print(f"   🛡️  Image appears authentic")
                else:
                    # Calculate similarity
                    similarity = 0
                    min_len = min(len(expected_watermark), len(extracted_watermark.strip()))
                    for i in range(min_len):
                        if expected_watermark[i] == extracted_watermark[i]:
                            similarity += 1
                    
                    if min_len > 0:
                        similarity_percent = (similarity / min_len) * 100
                        print(f"   ❌ MISMATCH: {similarity_percent:.1f}% similarity")
                        print(f"   ⚠️  Image may have been tampered with!")
                    else:
                        print(f"   ❌ NO MATCH: Watermark corrupted")
                
                # Display the image being verified
                plt.figure(figsize=(8, 6))
                if image_mode == 'color':
                    plt.imshow(verify_image)
                else:
                    plt.imshow(verify_image, cmap='gray')
                plt.title('🔍 Image Being Verified')
                plt.axis('off')
                plt.tight_layout()
                plt.show()
                
                # Test fragility by showing what happens with minor changes
                print(f"\n🧪 FRAGILITY TEST:")
                tampered = verify_image.copy()
                if len(tampered.shape) == 3:
                    tampered[100:120, 100:120, :] = 128  # Small gray square
                else:
                    tampered[100:120, 100:120] = 128
                
                tampered_extracted = extract_watermark_lsb(tampered, watermark_length, image_mode)
                fragility_test = expected_watermark not in tampered_extracted
                print(f"   Watermark breaks after minor modification: {'✅ YES' if fragility_test else '❌ NO'}")
                
            except Exception as e:
                print(f"❌ Error during verification: {str(e)}")
            finally:
                # Cleanup
                if os.path.exists(temp_image_path):
                    os.remove(temp_image_path)
                if os.path.exists('watermark_info.json'):
                    os.remove('watermark_info.json')
    
    verify_btn.on_click(verify_watermark)
    
    return widgets.VBox([
        widgets.HTML("<h2>🔍 Phase 2: Verify Watermark</h2>"),
        widgets.HTML("<p>Upload a watermarked image to verify its authenticity</p>"),
        verify_upload,
        info_upload,
        manual_watermark,
        verify_btn,
        output_area
    ])

In [9]:
def get_uploaded_content(upload_widget):
    """Universal method to extract content from upload widget"""
    if not upload_widget.value:
        return None
    upload_value = upload_widget.value
    
    try:
        if isinstance(upload_value, dict):
            return list(upload_value.values())[0]['content']
        elif isinstance(upload_value, (tuple, list)) and upload_value:
            first_file = upload_value[0]
            if isinstance(first_file, dict) and 'content' in first_file:
                return first_file['content']
            else:
                return list(first_file.values())[0]['content']
    except Exception as e:
        print(f"❌ Upload error: {e}")
    return None

In [10]:
# 5. Main Interface with Tabs
def create_main_interface():
    """Create the main interface with separate embedding and verification tabs"""
    
    # Create tab for embedding
    embedding_tab = create_embedding_interface()
    
    # Create tab for verification
    verification_tab = create_verification_interface()
    
    # Create tabs container
    tab = widgets.Tab()
    tab.children = [embedding_tab, verification_tab]
    tab.titles = ['🔒 Embed Watermark', '🔍 Verify Watermark']
    
    # Display main interface
    display(widgets.VBox([
        widgets.HTML("<h1>🔒 Fragile Watermarking System</h1>"),
        widgets.HTML("<p>Separate embedding and verification phases for better workflow</p>"),
        tab
    ]))

In [11]:
# 6. Run the System
print("✅ System initialized with separate embedding and verification!")
create_main_interface()

✅ System initialized with separate embedding and verification!
